# Advanced SQL Concepts

This notebook covers advanced SQL topics that build upon the fundamentals. You'll learn powerful features used in real-world data analysis and database management.

## Topics Covered:
1. Window Functions
2. Common Table Expressions (CTEs)
3. Advanced Joins (Self-joins, Cross joins)
4. Views and Indexes
5. Data Types and Constraints
6. Performance Optimization
7. Advanced Aggregations
8. Conditional Logic

## Setup and Database Connection

Let's connect to our existing database and ensure we have the sample data.

In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display
import os

# Connect to the SQLite database
if os.path.exists('my_database.db'):
    conn = sqlite3.connect('my_database.db')
elif os.path.exists('../my_database.db'):
    conn = sqlite3.connect('../my_database.db')
else:
    conn = sqlite3.connect('my_database.db')

cursor = conn.cursor()

print("Connected to SQLite database successfully!")
print(f"Database location: {os.path.abspath(conn.execute('PRAGMA database_list').fetchone()[2])}")

# Verify our tables exist
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(f"\nAvailable tables: {', '.join(tables['name'].tolist())}")

## 1. Window Functions

Window functions perform calculations across a set of rows related to the current row, without grouping the result set.

### Key Window Functions:
- **ROW_NUMBER()** - Assigns unique numbers to rows
- **RANK()** - Assigns ranks with gaps for ties
- **DENSE_RANK()** - Assigns ranks without gaps
- **LAG()/LEAD()** - Access previous/next row values
- **FIRST_VALUE()/LAST_VALUE()** - Get first/last values in window

In [ ]:
# Example 1: ROW_NUMBER() - Number employees by salary within each department
print("Employee ranking by salary within each department:")
query = """
SELECT 
    first_name,
    last_name,
    salary,
    dept_id,
    ROW_NUMBER() OVER (PARTITION BY dept_id ORDER BY salary DESC) as salary_rank
FROM employees
ORDER BY dept_id, salary_rank
"""
df = pd.read_sql_query(query, conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: RANK() and DENSE_RANK() comparison
print("Comparison of RANK() vs DENSE_RANK():")
query = """
SELECT 
    first_name,
    last_name,
    salary,
    RANK() OVER (ORDER BY salary DESC) as rank_with_gaps,
    DENSE_RANK() OVER (ORDER BY salary DESC) as dense_rank,
    ROW_NUMBER() OVER (ORDER BY salary DESC) as row_number
FROM employees
ORDER BY salary DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

In [ ]:
# Example 3: LAG() and LEAD() - Compare with previous/next values
print("Salary comparison with previous employee:")
query = """
SELECT 
    first_name,
    last_name,
    salary,
    LAG(salary, 1) OVER (ORDER BY salary) as prev_salary,
    salary - LAG(salary, 1) OVER (ORDER BY salary) as salary_diff
FROM employees
ORDER BY salary
"""
df = pd.read_sql_query(query, conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 4: Running totals with SUM() OVER()
print("Running total of salaries by department:")
query = """
SELECT 
    e.first_name,
    e.last_name,
    d.dept_name,
    e.salary,
    SUM(e.salary) OVER (PARTITION BY e.dept_id ORDER BY e.salary) as running_total
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
ORDER BY d.dept_name, e.salary
"""
df = pd.read_sql_query(query, conn)
display(df)

## 2. Common Table Expressions (CTEs)

CTEs allow you to define temporary result sets that can be referenced within a SELECT, INSERT, UPDATE, or DELETE statement.

In [ ]:
# Example 1: Basic CTE - Calculate department averages
print("Employees above their department average salary:")
query = """
WITH dept_averages AS (
    SELECT 
        dept_id,
        AVG(salary) as avg_salary
    FROM employees
    GROUP BY dept_id
)
SELECT 
    e.first_name,
    e.last_name,
    e.salary,
    d.dept_name,
    da.avg_salary,
    ROUND(e.salary - da.avg_salary, 2) as above_avg
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
JOIN dept_averages da ON e.dept_id = da.dept_id
WHERE e.salary > da.avg_salary
ORDER BY above_avg DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Multiple CTEs - Complex analysis
print("Department performance analysis:")
query = """
WITH dept_stats AS (
    SELECT 
        dept_id,
        COUNT(*) as employee_count,
        AVG(salary) as avg_salary,
        MAX(salary) as max_salary,
        MIN(salary) as min_salary
    FROM employees
    GROUP BY dept_id
),
project_budgets AS (
    SELECT 
        dept_id,
        COUNT(*) as project_count,
        SUM(budget) as total_budget
    FROM projects
    GROUP BY dept_id
)
SELECT 
    d.dept_name,
    d.location,
    ds.employee_count,
    ROUND(ds.avg_salary, 0) as avg_salary,
    pb.project_count,
    pb.total_budget,
    ROUND(pb.total_budget / ds.employee_count, 0) as budget_per_employee
FROM departments d
LEFT JOIN dept_stats ds ON d.dept_id = ds.dept_id
LEFT JOIN project_budgets pb ON d.dept_id = pb.dept_id
ORDER BY budget_per_employee DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

## 3. Advanced Joins

Beyond basic INNER and LEFT joins, let's explore self-joins and cross joins.

In [ ]:
# Example 1: Self-join - Find employees in the same department
print("Employees working in the same department:")
query = """
SELECT 
    e1.first_name || ' ' || e1.last_name as employee1,
    e2.first_name || ' ' || e2.last_name as employee2,
    d.dept_name,
    d.location
FROM employees e1
JOIN employees e2 ON e1.dept_id = e2.dept_id AND e1.emp_id < e2.emp_id
JOIN departments d ON e1.dept_id = d.dept_id
ORDER BY d.dept_name, employee1
"""
df = pd.read_sql_query(query, conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Cross join for combinations (be careful with large datasets!)
print("All possible employee-project combinations (first 10):")
query = """
SELECT 
    e.first_name || ' ' || e.last_name as employee,
    p.project_name,
    CASE 
        WHEN e.dept_id = p.dept_id THEN 'Assigned Department'
        ELSE 'Other Department'
    END as assignment_type
FROM employees e
CROSS JOIN projects p
ORDER BY employee, p.project_name
LIMIT 10
"""
df = pd.read_sql_query(query, conn)
display(df)

## 4. Views and Indexes

Views create virtual tables based on queries, while indexes improve query performance.

In [ ]:
# Example 1: Create a view for employee details
print("Creating and using a view:")
cursor.execute("""
CREATE VIEW IF NOT EXISTS employee_details AS
SELECT 
    e.emp_id,
    e.first_name || ' ' || e.last_name as full_name,
    e.email,
    e.salary,
    d.dept_name,
    d.location,
    CASE 
        WHEN e.salary >= 90000 THEN 'Senior'
        WHEN e.salary >= 75000 THEN 'Mid-level'
        ELSE 'Junior'
    END as level
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
""")

# Query the view
df = pd.read_sql_query("SELECT * FROM employee_details ORDER BY salary DESC", conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Create an index for better performance
print("Creating index on salary for faster queries:")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_employee_salary ON employees(salary)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_employee_dept ON employees(dept_id)")

# Show index information
indexes = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='index' AND tbl_name='employees'", conn)
print("Indexes on employees table:")
display(indexes)

conn.commit()

## 5. Advanced Conditional Logic

Using CASE statements and conditional aggregations for complex business logic.

In [ ]:
# Example 1: Complex CASE statements
print("Employee categorization with multiple conditions:")
query = """
SELECT 
    first_name || ' ' || last_name as employee,
    salary,
    hire_date,
    CASE 
        WHEN salary >= 90000 AND hire_date < '2020-01-01' THEN 'Senior Veteran'
        WHEN salary >= 90000 THEN 'Senior New Hire'
        WHEN salary >= 75000 AND hire_date < '2020-01-01' THEN 'Mid-level Veteran'
        WHEN salary >= 75000 THEN 'Mid-level New Hire'
        ELSE 'Junior Level'
    END as employee_category,
    CASE 
        WHEN salary > (SELECT AVG(salary) FROM employees) THEN 'Above Average'
        ELSE 'Below Average'
    END as salary_comparison
FROM employees
ORDER BY salary DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

print("\n" + "="*50 + "\n")

# Example 2: Conditional aggregations (pivot-like behavior)
print("Department salary distribution:")
query = """
SELECT 
    d.dept_name,
    COUNT(*) as total_employees,
    SUM(CASE WHEN e.salary >= 90000 THEN 1 ELSE 0 END) as senior_count,
    SUM(CASE WHEN e.salary BETWEEN 75000 AND 89999 THEN 1 ELSE 0 END) as mid_count,
    SUM(CASE WHEN e.salary < 75000 THEN 1 ELSE 0 END) as junior_count,
    ROUND(AVG(CASE WHEN e.salary >= 90000 THEN e.salary END), 0) as avg_senior_salary
FROM departments d
LEFT JOIN employees e ON d.dept_id = e.dept_id
GROUP BY d.dept_id, d.dept_name
ORDER BY total_employees DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

## 6. Practice Exercises - Advanced Level

Test your understanding with these challenging exercises!

### Advanced Exercise Questions:

1. **Window Function Challenge**: Find the 2nd highest paid employee in each department
2. **CTE Challenge**: Calculate the percentage of total company salary each department represents
3. **Self-Join Challenge**: Find pairs of employees with salary differences less than $5,000
4. **Performance Analysis**: Create a view showing project ROI (budget per employee in department)
5. **Complex Aggregation**: Show month-over-month hiring trends (extract month from hire_date)

In [ ]:
# Advanced Practice Area - Try the exercises above!

# Solution 1: 2nd highest paid employee in each department
print("Solution 1: 2nd highest paid employee in each department")
query1 = """
SELECT 
    first_name,
    last_name,
    salary,
    dept_id,
    salary_rank
FROM (
    SELECT 
        first_name,
        last_name,
        salary,
        dept_id,
        RANK() OVER (PARTITION BY dept_id ORDER BY salary DESC) as salary_rank
    FROM employees
) ranked
WHERE salary_rank = 2
"""
df = pd.read_sql_query(query1, conn)
display(df)

print("\n" + "="*50 + "\n")

# Solution 2: Department salary percentage of total
print("Solution 2: Department salary as percentage of total company")
query2 = """
WITH company_total AS (
    SELECT SUM(salary) as total_salary FROM employees
),
dept_totals AS (
    SELECT 
        dept_id,
        SUM(salary) as dept_salary
    FROM employees
    GROUP BY dept_id
)
SELECT 
    d.dept_name,
    dt.dept_salary,
    ct.total_salary,
    ROUND((dt.dept_salary * 100.0 / ct.total_salary), 2) as percentage_of_total
FROM departments d
JOIN dept_totals dt ON d.dept_id = dt.dept_id
CROSS JOIN company_total ct
ORDER BY percentage_of_total DESC
"""
df = pd.read_sql_query(query2, conn)
display(df)

# Add your solutions for exercises 3-5 here!

In [ ]:
# Cleanup
print("Advanced SQL concepts completed!")
print("Remember to practice these concepts with your own datasets.")
print("Next: Try the Data Analysis with SQL notebook for real-world applications!")

# Note: Keep connection open for other notebooks
# conn.close() would be called at the very end

In [ ]:
# Cleanup
print("Advanced SQL concepts completed!")
print("Remember to practice these concepts with your own datasets.")
print("Next: Try the Data Analysis with SQL notebook for real-world applications!")

# Note: Keep connection open for other notebooks
# conn.close() would be called at the very end